Smith Waterman is almost the same thing as Neeleman Wunsch.
First we define a couple of functions that we will need later, for formating and printing alignments. Here we have to modify the function `format_alignment` a bit so that it traces the optimal path from the maximal element to the first score zero.

In [63]:
import argparse
import numpy as np

def print_alignment(seqA,seqB):
  print(seqA)
  print(seqB)

def print_dynamic(seqA,seqB,dpm):
  seqA,seqB = "-" + seqA, "-" + seqB
  m,n = len(seqA),len(seqB)
  print '{:^5}'.format(" "),
  for j in range(n):
    print '{:^5}'.format(seqB[j]),
  print
  for i in range(m):
    print '{:^5}'.format(seqA[i]),
    for j in range(n):
        print '{:5.1f}'.format(dpm[i,j]),
    print
  print

def format_alignment(seqA,seqB,score,trace):
    outA,outB = "",""
    i,j = np.unravel_index(score.argmax(),score.shape)
    di,dj = trace[i,j]
    while di<0. or dj<0.:
        i += int(di)
        j += int(dj)
        if di == 0:
            outA = "-" + outA
        else:
            outA = seqA[i] + outA
        if dj == 0:
            outB = "-" + outB
        else:
            outB = seqB[j] + outB
        di,dj = trace[i,j]
    return outA,outB

Then we setup the scoring system we need for the alignment

In [64]:
gap_penalty = -1.0

def match_score(letterA,letterB):
    if letterA == letterB:
        return 1.0
    else:
        return -1.0

Now we turn our attention to the core of the Needleman-Wunsch, the dynamic programming

In [65]:
def align(seqA,seqB,print_dynamic_matrix = False):
    # Initiating variables
    m, n = len(seqA)+1, len(seqB)+1
    S = np.zeros((m,n))
    trace = np.zeros((m,n,2))
    # Set up dynamic programming matrices
    S[0,0] = 0.
    trace[0,0,:] = (0.,0.)
    for i in range(1,m):
        S[i,0] = 0.
        trace[i,0,:] =(0,0)
    for j in range(1,n):
        S[0,j] = 0.
        trace[0,j,:] =(0,0)
    for i in range(1,m):
        for j in range(1,n):
            # Note the subtraction of 1 for the sequence position
            #
            match = S[i-1,j-1] + match_score(seqA[i-1],seqB[j-1]) 
            delete = S[i-1,j] + gap_penalty
            insert = S[i,j-1] + gap_penalty
            S[i,j] = max(match,delete,insert,0.)
            if 0. >= max(match,insert,delete):
                trace[i,j,:] = (0,0)
            elif match >= max(insert,delete):
                trace[i,j,:] = (-1,-1)
            elif delete >= insert:
                trace[i,j,:] = (-1,0)
            else:
                trace[i,j,:] = (0,-1)
    if print_dynamic_matrix:
        print_dynamic(seqA,seqB,S)
    print("Best score: " + str(np.max(S)))
    return format_alignment(seqA,seqB,S,trace)

Now everything is set. We can try the code for any of our favorite sequences. One can toggle the printout of the dynamic programming matrix by a boolean flag as a third argument.

In [66]:
seqA,seqB = align("GATTA","GCTAC",True)
print_alignment(seqA,seqB)

        -     G     C     T     A     C  
  -     0.0   0.0   0.0   0.0   0.0   0.0
  G     0.0   1.0   0.0   0.0   0.0   0.0
  A     0.0   0.0   0.0   0.0   1.0   0.0
  T     0.0   0.0   0.0   1.0   0.0   0.0
  T     0.0   0.0   0.0   1.0   0.0   0.0
  A     0.0   0.0   0.0   0.0   2.0   1.0

Best score: 2.0
TA
TA


Here are a couple of extra alignments, check the dynamic programming matrices manually as an excercise before the exam.

In [67]:
seqA,seqB = align("GCGATTA","GCTTAC",True)
print_alignment(seqA,seqB)

        -     G     C     T     T     A     C  
  -     0.0   0.0   0.0   0.0   0.0   0.0   0.0
  G     0.0   1.0   0.0   0.0   0.0   0.0   0.0
  C     0.0   0.0   2.0   1.0   0.0   0.0   1.0
  G     0.0   1.0   1.0   1.0   0.0   0.0   0.0
  A     0.0   0.0   0.0   0.0   0.0   1.0   0.0
  T     0.0   0.0   0.0   1.0   1.0   0.0   0.0
  T     0.0   0.0   0.0   1.0   2.0   1.0   0.0
  A     0.0   0.0   0.0   0.0   1.0   3.0   2.0

Best score: 3.0
TTA
TTA


In [68]:
seqA,seqB = align("GCTATCTCGCTA","GCTAGCTA",True)
print_alignment(seqA,seqB)

        -     G     C     T     A     G     C     T     A  
  -     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  G     0.0   1.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0
  C     0.0   0.0   2.0   1.0   0.0   0.0   2.0   1.0   0.0
  T     0.0   0.0   1.0   3.0   2.0   1.0   1.0   3.0   2.0
  A     0.0   0.0   0.0   2.0   4.0   3.0   2.0   2.0   4.0
  T     0.0   0.0   0.0   1.0   3.0   3.0   2.0   3.0   3.0
  C     0.0   0.0   1.0   0.0   2.0   2.0   4.0   3.0   2.0
  T     0.0   0.0   0.0   2.0   1.0   1.0   3.0   5.0   4.0
  C     0.0   0.0   1.0   1.0   1.0   0.0   2.0   4.0   4.0
  G     0.0   1.0   0.0   0.0   0.0   2.0   1.0   3.0   3.0
  C     0.0   0.0   2.0   1.0   0.0   1.0   3.0   2.0   2.0
  T     0.0   0.0   1.0   3.0   2.0   1.0   2.0   4.0   3.0
  A     0.0   0.0   0.0   2.0   4.0   3.0   2.0   3.0   5.0

Best score: 5.0
GCTATCT
GCTAGCT
